In [30]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [82]:
data = pd.read_csv('../Resources/beauty_reviews.csv')
data

,reviewText,label
0,its up to you but as per the poppyaustin.com s...,0
1,I didn't have this item in my home for more th...,0
2,Description states there should be 144 tattoos...,0
3,"Agreed, this hair is nothing like what you act...",0
4,Not quite what I expected!,0
...,...,...
99995,This is very good for eczema and even more eff...,1
99996,Love the shimmer and a chap stick as well.,1
99997,Thanks!!!,1
99998,"Love it, and have been wearing this for 30 years",1


In [27]:
data['pre_process'] = data['reviewText'].apply(lambda x: ' '.join(x.lower() for x in str(x).split()))

In [28]:
from bs4 import BeautifulSoup
data['pre_process']=data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())

data['pre_process']=data['pre_process'].apply(lambda x: re.sub(r'http\S+', '', x))

In [68]:
def process_text(doc):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', doc)
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(w) for w in word_tokenize(re_clean)]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return ' '.join(output)
data['pre_process'] = data['pre_process'].apply(lambda x: process_text(x))

In [67]:
data

,reviewText,label,pre_process
0,Do Not deal with these people!\n True Derma is...,0,deal people true derma total scam product usel...
1,Disclored,0,disclored
2,I've had it for 2 months and two brushes broke...,0,ive month two brush broke good quality
3,It was horrible purchase..The product was defe...,0,wa horrible purchasethe product wa defective
4,"Okay, I've tried these twice now, and all they...",0,okay ive tried twice leave oily mess ive tried...
...,...,...,...
99995,This does exactly what I need to trim my beard...,1,doe exactly need trim beard even closer shave ...
99996,I have used Norelco razors for over 40 years. ...,1,used norelco razor year found le irritating el...
99997,I just LOVE this product. It really does the ...,1,love product really doe job feel much cleaner ...
99998,I started using this product one month ago. I...,1,started using product one month ago know take ...


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['pre_process'], data['label'], test_size=0.25, random_state=30)
print(f'Train: {X_train.shape,y_train.shape}\nTest: {X_test.shape,y_test.shape}')

Train: ((75000,), (75000,))
Test: ((25000,), (25000,))


In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_X_train = vectorizer.fit_transform(X_train)
tf_X_test = vectorizer.transform(X_test)

In [56]:
tf_X_train[0].toarray().sum()

5.966467953956153

In [57]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver='saga')

In [58]:
clf.fit(tf_X_train,y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [59]:
y_test_pred=clf.predict(tf_X_test)

In [61]:
from sklearn.metrics import classification_report
report = pd.DataFrame(classification_report(y_test_pred, 
                                        y_test, digits=2,
                                        output_dict=True)).T

report['support'] = report.support.apply(int)

report.style.background_gradient(cmap='RdYlGn',
                             subset=pd.IndexSlice['0':'9', :'f1-score'])

,precision,recall,f1-score,support
0,0.897185,0.880315,0.888670,12817
1,0.876529,0.893869,0.885114,12183
accuracy,0.886920,0.886920,0.886920,0
macro avg,0.886857,0.887092,0.886892,25000
weighted avg,0.887119,0.886920,0.886937,25000


In [62]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

In [64]:
clf.fit(tf_X_train,y_train)

LinearSVC(random_state=0)

In [65]:
y_test_pred=clf.predict(tf_X_test)

In [66]:
from sklearn.metrics import classification_report
report = pd.DataFrame(classification_report(y_test_pred, 
                                        y_test, digits=2,
                                        output_dict=True)).T

report['support'] = report.support.apply(int)

report.style.background_gradient(cmap='RdYlGn',
                             subset=pd.IndexSlice['0':'9', :'f1-score'])

,precision,recall,f1-score,support
0,0.889631,0.881987,0.885792,12685
1,0.879507,0.887292,0.883383,12315
accuracy,0.884600,0.884600,0.884600,0
macro avg,0.884569,0.884639,0.884587,25000
weighted avg,0.884644,0.884600,0.884605,25000


In [73]:
new_string = "This product is very bad and gross and I do not like it at all"

processed_new = process_text(new_string)

tfidf_new = vectorizer.transform([processed_new])

new_predict = clf.predict(tfidf_new)
new_predict

array([0])

In [72]:
new_string = "This product is very good and awesome and I am very happy with my purchase"

processed_new = process_text(new_string)

tfidf_new = vectorizer.transform([processed_new])

new_predict = clf.predict(tfidf_new)
new_predict

array([1])